In [1]:
!pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [2]:
!pip install -q streamlit openai requests wikipedia-api pyngrok fastapi uvicorn openai requests wikipedia-api

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.3 MB/s eta 0:00:00


In [3]:
!ngrok authtoken 2suOuuxs3zjz3pWQWDl9dQZPTLR_5FKMVwXpnfPJDKgESoGpK


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


## Wandertales FastAPI

In [ ]:
%%writefile fastapi_backend.py

from fastapi import FastAPI, Query, HTTPException
import openai
import requests
import wikipediaapi
from pyngrok import ngrok
import uvicorn
import nest_asyncio

nest_asyncio.apply()

OPENAI_API_KEY = "your_apenapikey"
# SerpAPI Key (Google Search)
SERPAPI_KEY = "your_serapikey"

# Initialize OpenAI API Client
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# Initialize FastAPI app
app = FastAPI()

# Wikipedia API Setup
wiki = wikipediaapi.Wikipedia(user_agent="WanderTales/1.0", language="en")

# Root Endpoint
@app.get("/")
async def root():
    return {"message": "Welcome to WanderTales API! Use /generate_travel_story/?destination=CityName or /generate_travel_image/?destination=CityName."}

# Fetch travel info from Wikipedia
def get_travel_info(destination: str):
    page = wiki.page(destination)
    return page.summary[:500] if page.exists() else "No travel information found."

# Fetch travel news from SerpAPI
def fetch_travel_news(destination: str):
    try:
        url = f"https://serpapi.com/search.json?q={destination}+travel+news&api_key={SERPAPI_KEY}"
        response = requests.get(url)
        results = response.json().get("organic_results", [])
        return results[0]["snippet"] if results else "No real-time travel news available."
    except Exception as e:
        return f"Error fetching travel news: {str(e)}"

# Generate travel story using OpenAI GPT-4
@app.get("/generate_travel_story/")
async def generate_travel_story(destination: str = Query(..., description="Enter a valid travel destination")):
    if not destination:
        raise HTTPException(status_code=400, detail="Destination parameter is required.")

    wikipedia_info = get_travel_info(destination)
    news_info = fetch_travel_news(destination)

    prompt = f"""
    You are a travel AI expert. Provide a captivating travel story for {destination}.

    - Historical & Cultural Insights: {wikipedia_info}
    - Latest Travel News: {news_info}

    Include famous landmarks, local experiences, and hidden gems.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an AI travel assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500
        )
        story = response.choices[0].message.content.strip()
        return {"story": story}

    except openai.OpenAIError as e:
        raise HTTPException(status_code=500, detail=f"OpenAI API error: {str(e)}")

# Generate AI-powered travel image using OpenAI DALL·E
@app.get("/generate_travel_image/")
async def generate_travel_image(destination: str = Query(..., description="Enter a valid travel destination")):
    if not destination:
        raise HTTPException(status_code=400, detail="Destination parameter is required.")

    try:
        response = client.images.generate(
            model="dall-e-3",  # Specify DALL·E model
            prompt=f"Scenic view of {destination} including cultural and historical landmarks",
            n=1,
            size="1024x1024"
        )

        image_url = response.data[0].url
        return {"image_url": image_url}

    except openai.OpenAIError as e:
        raise HTTPException(status_code=500, detail=f"OpenAI API error: {str(e)}")

# 🔹 Start Ngrok for FastAPI on Port 8000
public_url = ngrok.connect(8000)
print(f"🚀 FastAPI Public URL: {public_url}")

# Run FastAPI Server
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


Overwriting fastapi_backend.py


In [ ]:
!python fastapi_backend.py

🚀 FastAPI Public URL: NgrokTunnel: "https://9251-35-201-245-191.ngrok-free.app" -> "http://localhost:8000"
INFO:     Started server process [2824]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     2605:a601:aecb:9d00:9c41:e8a5:333f:85db:0 - "GET / HTTP/1.1" 200 OK
INFO:     2605:a601:aecb:9d00:9c41:e8a5:333f:85db:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2605:a601:aecb:9d00:9c41:e8a5:333f:85db:0 - "GET /generate_travel_story/?destination=Paris HTTP/1.1" 200 OK
INFO:     136.32.202.125:0 - "GET /generate_travel_story/?destination=KansasCity HTTP/1.1" 200 OK
INFO:     136.32.202.125:0 - "GET /generate_travel_image/?destination=KansasCity HTTP/1.1" 200 OK
INFO:     136.32.202.125:0 - "GET / HTTP/1.1" 200 OK
INFO:     136.32.202.125:0 - "GET /generate_travel_story/?destination=KansasCity HTTP/1.1" 200 OK
INFO:     136.32.202.125:0 - "GET /generate_travel_image/?d

## WanderTales : Streamlit

In [6]:
%%writefile wandertales_streamlit.py
import streamlit as st
import openai
import requests
import wikipediaapi
from pyngrok import ngrok

OPENAI_API_KEY = "your_apenapikey"
# SerpAPI Key (Google Search)
SERPAPI_KEY = "your_serapikey"


# Initialize OpenAI Client
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# Wikipedia API Setup
wiki = wikipediaapi.Wikipedia(user_agent="WanderTales/1.0", language="en")

# Function to fetch travel knowledge from Wikipedia
def get_travel_info(destination):
    st.info(f"🔍 Fetching historical & cultural info for: {destination}")
    page = wiki.page(destination)
    return page.summary[:500] if page.exists() else "No travel information found."

# Function to fetch real-time travel insights from SerpAPI
def fetch_travel_news(destination):
    st.info(f"🔍 Fetching latest travel news for: {destination}")

    try:
        url = f"https://serpapi.com/search.json?q={destination}+travel+news&api_key={SERPAPI_KEY}"
        response = requests.get(url)
        results = response.json().get("organic_results", [])

        return results[0]["snippet"] if results else "No real-time travel news available."

    except Exception as e:
        return f"Error fetching travel news: {str(e)}"

# Function to generate AI-powered travel stories
def generate_travel_story(destination):
    st.info(f"📝 Generating AI-powered travel story for: {destination}")

    wikipedia_info = get_travel_info(destination)
    news_info = fetch_travel_news(destination)

    prompt = f"""
    You are a travel AI expert. Provide a captivating travel story for {destination}.

    - Historical & Cultural Insights: {wikipedia_info}
    - Latest Travel News: {news_info}

    Include famous landmarks, local experiences, and hidden gems.
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an AI travel assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500
    )

    return response.choices[0].message.content.strip()

# Function to generate an AI-powered travel image
def generate_travel_image(destination):
    st.info(f"🖼️ Generating AI-powered travel image for: {destination}")

    try:
        response = client.images.generate(
            model="dall-e-3",  # Specify DALL·E model
            prompt=f"Scenic view of {destination} including cultural and historical landmarks",
            n=1,
            size="1024x1024"
        )

        image_url = response.data[0].url
        return image_url

    except openai.OpenAIError as e:
        return f"Error generating travel image: {str(e)}"

# Streamlit UI
st.title("✈️ WanderTales: AI-Powered Travel Insights")

destination = st.text_input("Enter a travel destination:")

if st.button("Generate Travel Story"):
    story = generate_travel_story(destination)
    st.subheader("📖 AI-Generated Travel Story")
    st.write(story)

if st.button("Get Latest Travel News"):
    news = fetch_travel_news(destination)
    st.subheader("📰 Latest Travel Insights")
    st.write(news)

if st.button("Generate AI Travel Image"):
    image_url = generate_travel_image(destination)
    if image_url and "http" in image_url:
        st.image(image_url, caption=f"AI-Generated Image of {destination}")
    else:
        st.error("Failed to generate travel image.")

Overwriting wandertales_streamlit.py


In [7]:
!streamlit run wandertales_streamlit.py & npx localtunnel --port 8501




⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.73.12.147:8501

⠸⠼⠴⠦your url is: https://giant-walls-hug.loca.lt
  Stopping...
^C
